In [88]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import pytesseract
import torch
import re
from transformers import DonutProcessor, VisionEncoderDecoderModel,VisionEncoderDecoderConfig
import pytorch_lightning as pl
import warnings
warnings.filterwarnings("ignore")

class xtrac():
    """
    Class for performing document extraction using a pre-trained model.
    """

    def __init__(self):
        """
        Initialize the xtrac object.

        Raises:
            Exception: If an error occurs during model initialization.
        """
        try:
            self.image_size = [1280, 960]
            self.max_length = 768
            
            self.config_path = "model_params/config.pt"
            self.processor_path ="model_params/processor.pt"
            self.model_path ="model_params/model.pt"
            
            self.config = torch.load(self.config_path)
            self.processor = torch.load(self.processor_path)
            self.model = VisionEncoderDecoderModel.from_pretrained(self.model_path, config = self.config)
            # Change the decoder_start_token_id to 0.
            
            print("initialization done")
        except Exception as e:
            print("Error occurred during model initialization:", str(e))

    def predict(self, image_path):
        """
        Perform document extraction on the given image.

        Args:
            image (PIL.Image.Image): The input image in PIL format.

        Returns:
            str: Extracted document information in JSON format.

        Raises:
            Exception: If an error occurs during prediction.
        """
        try:
            self.image = Image.open(image_path)
            self.rotated_image = self.image
            self.rotation_angle = 0

            try:
                self.text = pytesseract.image_to_osd(self.image)
                self.rotation_angle = int(re.search("(?<=Rotate: )\d+", self.text).group(0))
                self.rotated_image = self.image.rotate(-self.rotation_angle, expand=True)
            except pytesseract.pytesseract.TesseractError as e:
                # Error occurred during OCR, handle as needed
                print("OCR Error:", str(e))

            self.pixel_values = self.processor(images=self.rotated_image, return_tensors="pt").pixel_values
           
            self.task_prompt = "</s>"
            self.decoder_input_ids = self.processor.tokenizer(self.task_prompt,add_special_tokens=False,return_tensors="pt")["input_ids"]

            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            # self.device =  "cpu"
            self.model = self.model.to(self.device)

            # Adjust decoder_input_ids to match the batch size of pixel_values
            self.decoder_input_ids = self.decoder_input_ids.expand(self.pixel_values.shape[0], -1)
            self.outputs = self.model.generate(
                self.pixel_values.to(self.device),
                decoder_input_ids=self.decoder_input_ids.to(self.device),
                max_length=self.model.decoder.config.max_position_embeddings,
                early_stopping=True,
                pad_token_id=self.processor.tokenizer.pad_token_id,
                eos_token_id=self.processor.tokenizer.eos_token_id,
                use_cache=True,
                num_beams=1,
                bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
                return_dict_in_generate=True,
                output_scores=True,
            )
        
            self.predictions = []
            for seq in self.processor.tokenizer.batch_decode(self.outputs.sequences):
                seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(self.processor.tokenizer.pad_token, "")
                seq = re.sub(r"<.*?>", "", seq, count=1).strip()
                seq = re.sub(r"(?<=>)|(?=</s_)", "", seq, count=1).strip()
                seq = self.processor.token2json(seq)
                self.predictions.append(seq)
                
            self.ans = self.predictions[0]['text_sequence']
            self.pattern = r"<s_(\w+)([^<]+)</s_\1"
            self.matches = re.findall(self.pattern, self.ans)

            # Create a dictionary to store the extracted key-value pairs
            self.result_dict = {key.strip(): value.strip() for key, value in self.matches}

            # Define a mapping of keys to their desired labels
            self.key_mapping = {
                "payment_terms": "",
                "date": "",
                "currency": "",
                "country": "",
                "amount": "",
                "Rname": "",
                "Bname": ""
            }

            # Print the extracted and transformed key-value pairs
            for key, value in self.result_dict.items():
                if key in self.key_mapping:
                    self.key_mapping[key] =value
            
            return self.key_mapping
                    
        except Exception as e:
            print("Error occurred during prediction:", str(e))



# Load your image
# ins = xtrac()
# print(ins.predict(image_path))